# PA005: High Value Customer Identification(Insiders)

# <font color = 'red'> 0.0. Imports </font>

In [1]:
import os
import joblib
import re
import pickle
import s3fs
import sqlite3

import pandas as pd
import numpy as np
import seaborn as sns
import umap.umap_ as umap
import scipy.stats as st

from datetime import datetime
from matplotlib import pyplot as plt
from plotly import express as px
from sqlalchemy import create_engine

from sklearn import cluster as c
from sklearn import metrics as me
from sklearn import ensemble as en
from sklearn import preprocessing as pp
from sklearn import decomposition as dd
from sklearn import manifold as mn
from sklearn import mixture as mx

## Load dataset

In [2]:
# load data
df_raw = pd.read_csv('../data/raw/data.csv', encoding = 'ISO-8859-1')

# <font color = 'red'> 1.0. Descrição dos Dados </font>

In [3]:
df1 = df_raw.copy()

## 1.1. Rename columns

In [4]:
cols_new = ['invoice_no', 'stock_code', 'description', 
            'quantity', 'invoice_date', 'unit_price', 
            'customer_id', 'country']
df1.columns = cols_new

## 1.2. Data dimensions

In [5]:
print('number of rows: {}'.format(df1.shape[0]))
print('number of columns: {}'.format(df1.shape[1]))

number of rows: 541909
number of columns: 8


## 1.3. Data types

In [6]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [7]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5. Replace NA

In [8]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [9]:
# create reference
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000 + len(df_backup), 1)

# merge original df with reference df
df1 = pd.merge(df1, df_backup, on = 'invoice_no', how = 'left')

# coalesce -> combine_first()
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop extra columns
df1 = df1.drop(columns = ['customer_id_x', 'customer_id_y'], axis = 1)

In [10]:
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.6. Change dtypes

In [11]:
# invoice date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%m/%d/%Y %H:%M').dt.date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'])

# customer id
df1['customer_id'] = df1['customer_id'].astype('int64')
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,United Kingdom,17850


In [12]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

## 1.7. Descriptive statistics

In [13]:
num_attributes = df1.select_dtypes(include = ['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude = ['int64', 'float64', 'datetime64[ns]'])

### 1.7.1. Numerical Attributes

In [14]:
# central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion - desvio padrão, mínimo, máximo, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# concatenate
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


### 1.7.2. Categorical Attributes

#### Invoice No

In [15]:
# invoices com letras e números -> isso é um problema!
# identificação
df_letter_invoices = df1.loc[df1['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x))), :]
df_letter_invoices.head()

print('Total number of invoices: {}'.format(len(df_letter_invoices)))
print('Total number of negative quantity: {}'.format(len(df_letter_invoices[df_letter_invoices['quantity'] < 0])))

Total number of invoices: 9291
Total number of negative quantity: 9288


#### Stock Code

In [16]:
# check stock codes only characters
df1.loc[df1['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))), 'stock_code'].unique()

array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

#### Description

In [17]:
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,United Kingdom,17850


#### Country

In [18]:
df1['country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [19]:
df1['country'].value_counts(normalize = True).head()

United Kingdom    0.914320
Germany           0.017521
France            0.015790
EIRE              0.015124
Spain             0.004674
Name: country, dtype: float64

In [20]:
df1[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending = False).head()

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31


# <font color = 'red'> 2.0. Filtragem de Variáveis </font>

In [21]:
df2 = df1.copy()

In [22]:
df2.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

In [23]:
# numerical attributes
df2 = df2.loc[df2['unit_price'] >= 0.04, :]

# categorical attributes
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

# description
df2 = df2.drop(columns='description', axis = 1)

# map
df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

# bad users
df2 = df2[~df2['customer_id'].isin([16446])]

# quantity
df2_returns = df2.loc[df1['quantity'] < 0, :]
df2_purchases = df2.loc[df1['quantity'] >= 0, :]

# <font color = 'red'> 3.0. Feature Engineering </font>

In [24]:
df3 = df2.copy()

## 3.1. Feature Creation

In [25]:
# data reference
df_ref = df3.drop(['invoice_no', 'stock_code', 'quantity', 
                'invoice_date', 'unit_price', 'country'], axis = 1).drop_duplicates(ignore_index = True)

In [26]:
df_ref.shape

(5786, 1)

### 3.1.1. Gross Revenue

In [27]:
# Gross Revenue (Faturamento) quantity * price
df2_purchases = df2_purchases.copy()
df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']

# Monetary
df_monetary = df2_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on = 'customer_id', how = 'left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
dtype: int64

### 3.1.2. Recency - Days from Last Purchase

In [28]:
# Recency - Last day purchase
df_recency = df2_purchases.loc[:, ['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge(df_ref, df_recency, on = 'customer_id', how = 'left' )
df_ref.isna().sum()


customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 3.1.3. Qty. Purchased

In [29]:
# Frequência de compra
df_freq = (df2_purchases.loc[:, ['customer_id', 'invoice_no']].drop_duplicates()
        .groupby( 'customer_id' )
        .count()
        .reset_index()
        .rename(columns = {'invoice_no': 'qty_invoices'}))
df_ref = pd.merge(df_ref, df_freq, on = 'customer_id', how = 'left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qty_invoices     91
dtype: int64

### 3.1.4. Total Qty. Items Purchased

In [30]:
# Numero de produtos
df_freq = (df2_purchases.loc[:, ['customer_id', 'quantity']].groupby('customer_id').sum()
                                                        .reset_index()
                                                        .rename(columns={'quantity': 'qty_items'}))
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qty_invoices     91
qty_items        91
dtype: int64

### 3.1.5. Qty. of Products Purchased

In [31]:
# Numero de produtos
df_freq = (df2_purchases.loc[:, ['customer_id', 'stock_code']].groupby('customer_id' ).count()
                                                        .reset_index()
                                                        .rename(columns={'stock_code': 'qty_products'}))
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qty_invoices     91
qty_items        91
qty_products     91
dtype: int64

### 3.1.6. Average Ticket Value

In [32]:
# Average Ticket
df_avg_ticket = (df2_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby('customer_id')
                .mean()
                .reset_index()
                .rename(columns = {'gross_revenue':'avg_ticket'}))
df_ref = pd.merge(df_ref, df_avg_ticket, on = 'customer_id', how = 'left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qty_invoices     91
qty_items        91
qty_products     91
avg_ticket       91
dtype: int64

### 3.1.7. Average Recency Days

In [33]:
# Average recency days
df_aux = df2[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'], ascending = [False, False])
df_aux['next_customer_id'] = df_aux['customer_id'].shift() # next customer
df_aux['previous_date'] = df_aux['invoice_date'].shift() # next invoince date

df_aux['avg_recency_days'] = df_aux.apply(lambda x: (x['invoice_date'] - x['previous_date']).days if x['customer_id'] == x['next_customer_id'] else np.nan, axis=1)

df_aux = df_aux.drop(['invoice_date', 'next_customer_id', 'previous_date'], axis=1).dropna()

# average recency 
df_avg_recency_days = df_aux.groupby('customer_id').mean().reset_index()

# merge
df_ref = pd.merge(df_ref, df_avg_recency_days, on = 'customer_id', how = 'left')
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qty_invoices          91
qty_items             91
qty_products          91
avg_ticket            91
avg_recency_days    2816
dtype: int64

### 3.1.8. Frequency Purchases

In [34]:
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                            .groupby( 'customer_id')
                                                            .agg(max_ = ('invoice_date', 'max'), 
                                                                min_ = ('invoice_date', 'min'),
                                                                days_= ('invoice_date', lambda x: ((x.max() - x.min()).days ) + 1),
                                                                buy_ = ('invoice_no', 'count'))).reset_index()

# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(lambda x: x['buy_'] / x['days_'] if  x['days_'] != 0 else 0, axis = 1)

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on = 'customer_id', how = 'left')
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qty_invoices          91
qty_items             91
qty_products          91
avg_ticket            91
avg_recency_days    2816
frequency             91
dtype: int64

### 3.1.9. Number of Returns

In [35]:
# returns
df_returns = df2_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'qty_returns'})
df_returns['qty_returns'] = df_returns['qty_returns'] * -1

df_ref = pd.merge(df_ref, df_returns, how = 'left', on = 'customer_id')
df_ref.loc[df_ref['qty_returns'].isna(), 'qty_returns'] = 0

df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qty_invoices          91
qty_items             91
qty_products          91
avg_ticket            91
avg_recency_days    2816
frequency             91
qty_returns            0
dtype: int64

# <font color = 'red'> 4.0. EDA (Exploratory Data Analysis) </font>

In [36]:
df4 = df_ref.dropna()

## 4.1. Estudo do Espaço

In [37]:
# df43 = df4.drop(columns = ['customer_id'], axis = 1).copy()
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 
                'qty_products', 'frequency', 'qty_returns']
df43 = df4[cols_selected].copy()

In [38]:
mm = pp.MinMaxScaler()
# fs = s3fs.S3FileSystem( anon=False, key=AWS_ACCESS_KEY_ID, secret=AWS_SECRET_ACCESS_KEY )

df43['gross_revenue'] = mm.fit_transform(df43[['gross_revenue']])
df43['recency_days'] = mm.fit_transform(df43[['recency_days']])
df43['qty_products'] = mm.fit_transform(df43[['qty_products']])
df43['frequency'] = mm.fit_transform(df43[['frequency']])
df43['qty_returns'] = mm.fit_transform(df43[['qty_returns']])

X = df43.copy()

## 4.2. Tree-Based Embedding

In [39]:
# training dataset
X = df43.drop(columns = ['customer_id', 'gross_revenue'], axis = 1)
y = df43['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor(n_estimators = 100, random_state = 42)

# model training
rf_model.fit(X, y)

# leaf
df_leaf = pd.DataFrame(rf_model.apply(X))

In [40]:
# Dimensionality reducer
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(df_leaf)

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

# <font color = 'red'> 5.0. Hyperparameter Fine-Tunning </font>

In [41]:
X = df_tree.copy()

In [42]:
X.head()

,embedding_x,embedding_y
0,10.918785,14.602229
1,19.531637,8.913908
2,11.945412,14.611155
3,0.142954,2.149642
4,-6.805303,9.022922


# <font color = 'red'> 6.0. Model Training </font>

## 6.1. Final Model

In [43]:
# model definition
k = 10

## model definition
gmm_model = mx.GaussianMixture(n_components = k, n_init = 300, random_state = 32)

# model training
gmm_model.fit(X)

# model predict
labels = gmm_model.predict(X)

## 6.2. Cluster Validation

In [44]:
# # WSS (Within-Cluster Sum of Square)
# print('WSS value: {}'.format(kmeans.inertia_))

# SS (Silhouette Score)
print('SS value: {}'.format(me.silhouette_score(X, labels, metric = 'euclidean')))

SS value: 0.6150678396224976


# <font color = 'red'> 7.0. Cluster Analysis </font>

In [45]:
df72 = df4[cols_selected].copy()
df72['cluster'] = labels

# change dtypes
df72['recency_days'] = df72['recency_days'].astype('int64')
df72['qty_products'] = df72['qty_products'].astype('int64')
df72['qty_returns'] = df72['qty_returns'].astype('int64')
df72['last_training_timestamp'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S')

In [46]:
df72.head()

,customer_id,gross_revenue,recency_days,qty_products,frequency,qty_returns,cluster,last_training_timestamp
0,17850,5391.21,372,297,17.000000,40,4,2023-03-20 11:33:40
1,13047,3232.59,56,171,0.028302,35,1,2023-03-20 11:33:40
2,12583,6705.38,2,232,0.040323,50,4,2023-03-20 11:33:40
3,13748,948.25,95,28,0.017921,0,0,2023-03-20 11:33:40
4,15100,876.00,333,3,0.073171,22,7,2023-03-20 11:33:40


In [47]:
# Number of customer
df_cluster = df72[['customer_id', 'cluster']].groupby( 'cluster' ).count().reset_index()
df_cluster['perc_customer'] = 100*(df_cluster['customer_id'] / df_cluster['customer_id'].sum())

# Avg Gross revenue
df_avg_gross_revenue = df72[['gross_revenue', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# Avg recency days
df_avg_recency_days = df72[['recency_days', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')

# Avg invoice_no
df_qty_products = df72[['qty_products', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge(df_cluster, df_qty_products, how='inner', on='cluster')

# Frequency
df_frequency = df72[['frequency', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge(df_cluster, df_frequency, how='inner', on='cluster')

# Returns
df_qty_returns = df72[['qty_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_qty_returns, how='inner', on='cluster')

df_cluster.sort_values( 'gross_revenue', ascending=False )

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qty_products,frequency,qty_returns
4,4,373,12.567385,10503.060483,19.268097,475.581769,0.110615,187.120643
1,1,501,16.880054,2624.627305,40.948104,141.542914,0.047552,20.940120
9,9,95,3.200809,2291.266000,29.957895,222.136842,0.031576,0.663158
6,6,349,11.758760,2216.344413,69.501433,52.896848,0.072810,11.028653
5,5,404,13.611860,1689.182401,54.801980,89.202970,0.050037,11.542079
8,8,107,3.605121,1448.545701,65.364486,66.878505,0.066564,33.186916
2,2,372,12.533693,1237.753145,61.282258,54.357527,0.043705,8.771505
0,0,175,5.896226,827.478286,92.022857,26.822857,0.161952,22.868571
3,3,200,6.738544,647.933500,47.280000,11.510000,0.025991,0.845000
7,7,392,13.207547,505.728827,149.426020,15.522959,0.426596,9.443878


In [48]:
# ================ REPENSAR ESSAS CATEGORIAS ============== #
# 4 Cluster Insiders
# 1 Cluster More Products
# 9 Cluster Spend Money 
# 6 Cluster Even More Products
# 5 Cluster Less Days
# 8 Cluster Less 1k
# 2 Cluster Stop Returners
# 0 Cluster More Buy
# 3
# 7

### Cluster Cadidato a Insider

- Número de customers: 292 (9% do customers )
- Faturamento médio: 15086
- Recência média: 22 dias
- Média de Produtos comprados: 436 produtos
- Frequência de Produtos comprados: 0.08 produtos/dia
- Receita em média: US$15086.34 dólares

# <font color = 'red'> 8.0. Deploy to Production </font>

In [49]:
df72.dtypes

customer_id                  int64
gross_revenue              float64
recency_days                 int64
qty_products                 int64
frequency                  float64
qty_returns                  int64
cluster                      int64
last_training_timestamp     object
dtype: object

## 11.1 Insert into SQLite

In [50]:
df72['recency_days'] = df72['recency_days'].astype('int64')
df72['qty_products'] = df72['qty_products'].astype('int64')
df72['qty_returns'] = df72['qty_returns'].astype('int64')

In [58]:
host = 'sqlite:///insiders_db.sqlite'
conn = create_engine(host)

In [60]:
# create table
#query_create_table_insiders = """
#CREATE TABLE insiders (
#    customer_id INTEGER,
#    gross_revenue REAL,
#    recency_days INTEGER,
#    qty_products INTEGER,
#    frequency REAL,
#    qty_returns REAL,
#    cluster INTEGER,
#    last_training_timestamp TIMESTAMP
#)
#"""
conn = sqlite3.connect('insiders_db.sqlite')
# conn.execute(query_create_table_insiders)
# conn.commit()

# insert data into table
df72.to_sql('insiders', con = conn, if_exists = 'append', index = False)

2968

In [61]:
# select data
query = """
    SELECT * FROM insiders
"""
df = pd.read_sql_query(query, conn)

In [62]:
conn.close()

In [63]:
df.head()

,customer_id,gross_revenue,recency_days,qty_products,frequency,qty_returns,cluster,last_training_timestamp
0,17850,5391.21,372,297,17.000000,40.0,4,2023-03-20 11:33:40
1,13047,3232.59,56,171,0.028302,35.0,1,2023-03-20 11:33:40
2,12583,6705.38,2,232,0.040323,50.0,4,2023-03-20 11:33:40
3,13748,948.25,95,28,0.017921,0.0,0,2023-03-20 11:33:40
4,15100,876.00,333,3,0.073171,22.0,7,2023-03-20 11:33:40


In [64]:
df.shape

(5936, 8)